In [130]:
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
from tqdm import tqdm

dataset_name_list = [
    # 'SHREC19_r_pair',
    'FAUST_r_pair',
    'FAUST_a_pair',
    'SCAPE_r_pair',
    'SCAPE_a_pair',
]

for dataset_name in dataset_name_list:

    single_dataset, pair_dataset = data_loading.get_val_dataset(
        dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
    )

    off_pair_list = []

    for i in tqdm(range(len(pair_dataset))):
        
        data_i = pair_dataset[i]
        
        first_index = data_i['first']['id']
        second_index = data_i['second']['id']
        
        off_first = single_dataset.off_files[first_index]
        off_second = single_dataset.off_files[second_index]
        
        # print(first_index)
        # print(second_index)
        
        # print(off_first)
        # print(off_second)
        
        off_pair_list.append({
            'first': first_index.item(),
            'second': second_index.item(),
            'off_first': f'/home/s94zalek_hpc/shape_matching/{off_first}',
            'off_second': f'/home/s94zalek_hpc/shape_matching/{off_second}'
        })

    with open(f'/home/s94zalek_hpc/shape_matching/notebooks/release/evaluation_pairs/{dataset_name}.yaml', 'w') as f:
        yaml.dump(off_pair_list, f, sort_keys=False)


In [ ]:
scene = trimesh.Scene()

In [ ]:
import trimesh

scene.geometry.clear()

mesh_1 = trimesh.load('/home/s94zalek_hpc/shape_matching/data/SCAPE_a/off/mesh054.off')

mesh_2 = trimesh.load(
    '/home/s94zalek_hpc/shape_matching/data/FAUST_a/off/tr_reg_080.off'
)

verts_1 = mesh_1.vertices.copy()

# full_data['verts'][:, 0] = verts_first_raw[:, 0]
# full_data['verts'][:, 1] = verts_first_raw[:, 2]
# full_data['verts'][:, 2] = -verts_first_raw[:, 1]

# mesh_1.vertices[:, 0] = verts_1[:, 1]
# mesh_1.vertices[:, 1] = verts_1[:, 0]
# mesh_1.vertices[:, 2] = verts_1[:, 2]

verts_1_fixed = verts_1.copy()
verts_1_fixed[:, 0] = -verts_1[:, 1]
verts_1_fixed[:, 1] = verts_1[:, 0]
verts_1_fixed[:, 2] = verts_1[:, 2]

mesh_1 = trimesh.Trimesh(vertices=verts_1_fixed, faces=mesh_1.faces)


mesh_3 = trimesh.load(
    '/home/s94zalek_hpc/shape_matching/data/SHREC19_r/off/1.off'
)

scene.add_geometry(mesh_1)
scene.add_geometry(mesh_2)
scene.add_geometry(mesh_3)


scene.show()

In [158]:
print(mesh_1.area_faces.sum())
print(mesh_2.area_faces.sum())
print(mesh_3.area_faces.sum())

In [165]:
# for each mesh in /home/s94zalek_hpc/shape_matching/data/FAUST_a/off

import os

for off_file in os.listdir('/home/s94zalek_hpc/shape_matching/data/SCAPE_a/off'):
    mesh = trimesh.load(f'/home/s94zalek_hpc/shape_matching/data/SCAPE_a/off/{off_file}', process=False)
    
    # verts_1_fixed = verts_1.copy()
    # verts_1_fixed[:, 0] = -verts_1[:, 1]
    # verts_1_fixed[:, 1] = verts_1[:, 0]
    # verts_1_fixed[:, 2] = verts_1[:, 2]
    
    verts_fixed = mesh.vertices.copy()
    verts_fixed[:, 0] = -mesh.vertices[:, 1]
    verts_fixed[:, 1] = mesh.vertices[:, 0]
    verts_fixed[:, 2] = mesh.vertices[:, 2]
    
    mesh = trimesh.Trimesh(vertices=verts_fixed, faces=mesh.faces, process=False)
    # save the mesh
    mesh.export(f'/home/s94zalek_hpc/shape_matching/data/SCAPE_a/off/{off_file}')


In [ ]:
scene.geometry.clear()

mesh_new = trimesh.load('/home/s94zalek_hpc/shape_matching/data/SCAPE_a/off/mesh055.off')
mesh_old = trimesh.load('/home/s94zalek_hpc/shape_matching/data/SCAPE_a/off_old/mesh055.off')
mesh_old.vertices += [1, 0, 0]


scene.add_geometry(mesh_new)

scene.add_geometry(mesh_old)

scene.show()

In [ ]:
# for each mesh in /home/s94zalek_hpc/shape_matching/data/SCAPE_a/off

import os

scene.geometry.clear()

dataset = 'SCAPE_a'

for i, off_file in enumerate(os.listdir(f'/home/s94zalek_hpc/shape_matching/data/{dataset}/off')[:20]):
    
    mesh = trimesh.load(f'/home/s94zalek_hpc/shape_matching/data/{dataset}/off/{off_file}', process=False)
    
    mesh.vertices += [i, 0, 0]
    
    scene.add_geometry(mesh)
    
scene.show()
    

In [ ]:
mesh = trimesh.load('/home/s94zalek_hpc/shape_matching/data/SHREC19_r/off/12.off')
mesh.show()

In [191]:
bad_str = 'mesh/dsds.off'
bad_str.replace('/', '_')

# Transmatch

In [204]:
import yaml
from tqdm import tqdm
import os
import json

dataset_name_list = [
    'SHREC19_r_pair',
    # 'FAUST_r_pair',
    'FAUST_a_pair',
    'SCAPE_r_pair',
    'SCAPE_a_pair',
]

for dataset_name in dataset_name_list:
    
    
    path_transmatch = f'/home/s94zalek_hpc/baselines/transmatching/evaluation/datasets/{dataset_name}/data'
    
    if os.path.exists(path_transmatch):
        os.system(f'rm -r {path_transmatch}')
    
    os.makedirs(path_transmatch, exist_ok=True)
    

    single_dataset, pair_dataset = data_loading.get_val_dataset(
        dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
    )

    for i in tqdm(range(len(pair_dataset))):
        
        data_i = pair_dataset[i]
        
        first_index = data_i['first']['id']
        second_index = data_i['second']['id']
        
        off_first = single_dataset.off_files[first_index]
        off_second = single_dataset.off_files[second_index]
        
        
        path_i = f'{path_transmatch}/{i:03d}'
        os.makedirs(path_i, exist_ok=True)
        
        
        mesh_first = trimesh.load(off_first, process=False, validate=False)
        mesh_second = trimesh.load(off_second, process=False, validate=False)
        
        # mesh_first.vertices = mesh_first.vertices / mesh_first.area_faces.sum() ** 0.5
        
        assert 0.99 < mesh_first.area_faces.sum() < 1.01, f'mesh_first.area_faces.sum() = {mesh_first.area_faces.sum()}'
        assert 0.99 < mesh_second.area_faces.sum() < 1.01, f'mesh_second.area_faces.sum() = {mesh_second.area_faces.sum()}'
        
        mesh_first.export(f'{path_i}/A.off')
        mesh_second.export(f'{path_i}/B.off')

        meta_dict = {
            'id': {
                'A': first_index.item(),
                'B': second_index.item()
            }
        }
        
        with open(f'{path_i}/meta.json', 'w') as f:
            json.dump(meta_dict, f)
            
        # break
        
        
        
        
        
        # print(first_index)
        # print(second_index)
        
        # print(off_first)
        # print(off_second)
        
        # off_pair_list.append({
        #     'first': first_index.item(),
        #     'second': second_index.item(),
        #     'off_first': f'/home/s94zalek_hpc/shape_matching/{off_first}',
        #     'off_second': f'/home/s94zalek_hpc/shape_matching/{off_second}'
        # })

    # with open(f'/home/s94zalek_hpc/shape_matching/notebooks/release/evaluation_pairs/{dataset_name}.yaml', 'w') as f:
    #     yaml.dump(off_pair_list, f, sort_keys=False)


In [203]:
mesh_first.scale